# Support Sentinel 🛡️ - A Stack Whisper

[![Streamlit App](https://static.streamlit.io/badges/streamlit_badge_black_white.svg)](https://stake-whisper.streamlit.app/)
[![Python 3.9+](https://img.shields.io/badge/python-3.9+-blue.svg)](https://www.python.org/downloads/)
[![Google Cloud](https://img.shields.io/badge/Google_Cloud-4285F4?logo=googlecloud)](https://cloud.google.com/bigquery)

### Data Extract from `bigquery-public-data`

In [ ]:
%%bigquery

-- Create a new, manageable table with high-quality questions and their accepted answers.
CREATE OR REPLACE TABLE kaggle.stackoverflow_qa AS
WITH questions AS (
  SELECT
    id,
    title,
    body AS question_body,
    accepted_answer_id
  FROM
    `bigquery-public-data.stackoverflow.posts_questions`
  WHERE
    accepted_answer_id IS NOT NULL
    AND score > 5 -- Filter for questions the community found useful

),
answers AS (
  -- Select the corresponding answers
  SELECT
    id,
    body AS answer_body
  FROM
    `bigquery-public-data.stackoverflow.posts_answers`
)
SELECT
  q.id AS ticket_id,
  CONCAT(q.title, '\n\n', REGEXP_REPLACE(q.question_body, r'<[^>]*>', '')) AS problem_description
  REGEXP_REPLACE(a.answer_body, r'<[^>]*>', '') AS resolution_text
FROM
  questions q
JOIN
  answers a ON q.accepted_answer_id = a.id
;

In [ ]:
%%bigquery

-- Check your new table
SELECT * FROM `kaggle.stackoverflow_qa` LIMIT 10;

## Create model

In [ ]:
%%bigquery

CREATE OR REPLACE MODEL `kaggle.text_embedding_model`
REMOTE WITH CONNECTION `us.vertexai`
OPTIONS (ENDPOINT = 'text-embedding-004');


In [ ]:
%%bigquery

CREATE OR REPLACE MODEL `master-booster-469602-q2.kaggle.gemini`
REMOTE WITH CONNECTION `master-booster-469602-q2.us.vertexai`
OPTIONS (ENDPOINT = 'gemini-2.0-flash-001');


## Create embedding

In [ ]:
%%bigquery

CREATE OR REPLACE TABLE kaggle.stackoverflow_with_embeddings AS
SELECT
  ticket_id,
  problem_description,
  resolution_text,
  ml_generate_embedding_result AS embedding
FROM
  ML.GENERATE_EMBEDDING(
    MODEL `kaggle.text_embedding_model`,
    (
      SELECT
        ticket_id,
        problem_description,
        resolution_text,
        problem_description AS content
      FROM kaggle.stackoverflow_qa
    )
  );

### Create Vactor Indexing for Quick Processing

In [ ]:
%%bigquery

-- Create a vector index on your scaled-up embeddings table
CREATE OR REPLACE VECTOR INDEX `stackoverflow_1m_index`
ON `master-booster-469602-q2.kaggle.stackoverflow_with_embeddings`(embedding)
OPTIONS(index_type='IVF', distance_type='COSINE', ivf_options='{"num_lists": 4000}');

### Generate the answer from Vector Search result

In [ ]:
%%bigquery

DECLARE
  new_question STRING DEFAULT "My python script is using too much RAM when I read a big csv file. How can I fix this?";
WITH
  question_embedding AS (
  SELECT
    p.ml_generate_embedding_result AS embedding
  FROM
    ML.generate_embedding( MODEL `master-booster-469602-q2.kaggle.text_embedding_model`,
      (
      SELECT
        new_question AS content) ) AS p ),
  prompt_generation AS (
  SELECT
    CONCAT( 'You are a helpful expert Python programmer. Answer the following question based ONLY on the context provided. Provide a clear, actionable solution with code examples. \n\n', 'CONTEXT: \n', IFNULL(STRING_AGG(retrieved_solutions.base.resolution_text, '\n---\n'), 'No relevant context found.'), '\n\nQUESTION: ', new_question ) AS prompt
  FROM (
    SELECT
      base
    FROM
      VECTOR_SEARCH( TABLE `master-booster-469602-q2.kaggle.stackoverflow_with_embeddings`,
        'embedding',
        (
        SELECT
          embedding
        FROM
          question_embedding),
        top_k => 3 ) ) AS retrieved_solutions )
  -- Final query with the added max_output_tokens parameter
SELECT
  --ml_generate_text_result['content'] AS generated_answer,
  ml_generate_text_result.candidates[0].content.parts[0].text,
  ml_generate_text_result AS full_model_response -- Keep this to monitor results
FROM
  ML.GENERATE_TEXT( MODEL `master-booster-469602-q2.kaggle.gemini`,
    TABLE prompt_generation,
    STRUCT(
      8192 AS max_output_tokens,
      0.5 AS temperature,
      0.95 AS top_p ) );